#### 1. Establish a connection between Python and the Sakila database.

In [5]:
import pandas as pd # all time remember to import pandas library

psw = '.Aa654321.'

import mysql.connector

cnx = mysql.connector.connect(user = 'root',
                              password = psw,
                              host = 'localhost',
                              database = 'Sakila',
                              auth_plugin = 'mysql_native_password')

cursor = cnx.cursor()

cursor

#### 2.Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

- engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
- month: an integer representing the month for which rental data is to be retrieved.
- year: an integer representing the year for which rental data is to be retrieved.

The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [17]:
def rentals_month(engine,month,year):
    query = f"""
    SELECT * 
    FROM rental 
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """ # first of all we define the query we will do
    return pd.read_sql(query, engine)

adb = rentals_month(cnx,6,2005)

#### 3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()

In [ ]:
def rental_count_month(DataFrame):
    query = f"""
    SELECT * 
    FROM rental 
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """ # first of all we define the query we will do
    return pd.read_sql(query, engine)

#### 4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.